<a href="https://colab.research.google.com/github/dercodeKoenig/colab-KI/blob/main/copygan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content
!cp drive/MyDrive/gen_weights.h5 gen_weights.h5
!cp drive/MyDrive/disc_weights.h5 disc_weights.h5

/content


In [ ]:
%%writefile kaggle.json
{"username":"lutze3","key":"c1b954a36f5402b976973271cb154a2f"}

In [ ]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/kaggle.json
!kaggle datasets download -d rikotheboss/nppreprocessed
!unzip nppreprocessed.zip
!mkdir input
!mv npf* input/

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
import numpy as np
from tensorflow.keras import layers
import time
import cv2
from IPython import display
from threading import Thread
 
t0= time.time()
 
resume = True
gen_weights = "gen_weights.h5"
disc_weights = "disc_weights.h5"
datadir="input/"

In [ ]:
GLOBAL_BATCH_SIZE=16
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    GLOBAL_BATCH_SIZE=GLOBAL_BATCH_SIZE*8
except:
    strategy = tf.distribute.MirroredStrategy()
#display.clear_output()

In [ ]:
namest = os.listdir(datadir)
print(namest)
small_images_cartoon = np.load(datadir+namest[0])
test_imgs = small_images_cartoon[0:8]
test_imgs = np.array(test_imgs)
del small_images_cartoon
        
train_images_cartoon = 0
zlr=0
 
def load_next():
    global train_images_cartoon, small_images_cartoon, zlr
    while("npf" not in namest[zlr]):
        zlr+=1
        if(zlr==len(namest)):
            zlr=0
    
 
    
    train_images_cartoon = np.load(datadir+namest[zlr])
    #print("new data: ",train_images_cartoon.shape)
    train_images_cartoon = tf.keras.layers.experimental.preprocessing.RandomZoom((0,0.1),(0,0.1),fill_mode='constant',fill_value=1)(train_images_cartoon)
    train_images_cartoon = tf.keras.layers.experimental.preprocessing.RandomTranslation((-0.2,0.2), (-0.2,0.2),fill_mode='constant',fill_value=1)(train_images_cartoon)
        
    train_images_cartoon = tf.data.Dataset.from_tensor_slices((train_images_cartoon)).batch(GLOBAL_BATCH_SIZE) 
    train_images_cartoon = strategy.experimental_distribute_dataset(train_images_cartoon)

    zlr+=1
    if(zlr==len(namest)):
      zlr=0
 
print("load first files")
load_next()

In [ ]:
def make_generator_model():
    inputa=tf.keras.layers.Input(shape=(128,128,3))
    
    x = layers.Conv2D(64, 5)(inputa)
    x = layers.LeakyReLU()(x)
 
    x = layers.Conv2D(128, 3)(x)
    x = layers.LeakyReLU()(x)
 
    x = layers.MaxPooling2D(2)(x)
 
    x = layers.Conv2D(256, 3)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(256, 3)(x)
    x = layers.LeakyReLU()(x)
 
    x = layers.MaxPooling2D(2)(x)
    
    x = layers.Conv2D(256, 3)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(256, 3)(x)
    x = layers.LeakyReLU()(x)
 
    x = layers.MaxPooling2D(2)(x)
    
    x = layers.Conv2D(512, 3)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(512, 3)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.MaxPooling2D(2)(x)
    
    x = layers.Flatten()(x)
 
    x = layers.Dense(100)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Dense(8*8*128)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Reshape((8,8,128))(x)
 
 
    x = layers.Conv2D(500, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(500, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(500, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(500, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
 
    x = layers.Conv2DTranspose(400, (5, 5), strides=(2, 2),padding="same")(x)
    x = layers.LeakyReLU()(x)
 
    x = layers.Conv2D(400, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
 
    x = layers.Conv2D(400, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(400, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(400, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
 
    x = layers.Conv2DTranspose(300, (5, 5), strides=(2, 2),padding="same")(x)
    x = layers.LeakyReLU()(x)
 
    
    x = layers.Conv2D(300, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
 
    x = layers.Conv2D(300, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(300, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(300, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
 
    
    x = layers.Conv2DTranspose(200, (5, 5),padding="same", strides=(2, 2))(x)
    x = layers.LeakyReLU()(x)
 
    x = layers.Conv2D(200, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(200, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(200, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(200, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2DTranspose(100, (5, 5),padding="same", strides=(2, 2))(x)
    x = layers.LeakyReLU()(x)
 
    x = layers.Conv2D(100, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
 
    x = layers.Conv2D(100, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(100, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(100, 3,padding="same")(x)
    x = layers.LeakyReLU()(x)
    
    outputs = layers.Conv2D(3,3,padding="same",strides=(1, 1), activation='tanh')(x)
 
    model = tf.keras.Model(inputs=inputa, outputs=outputs, name="generator_model")
    return model
 
def make_discriminator_model():
    inputa = layers.Input(shape=(128,128,3))
 
    x = layers.Conv2D(50,7)(inputa)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)
 
    x = layers.Conv2D(100,5)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(100,5)(x)
    x = layers.Conv2D(100,5)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Conv2D(200,3,strides=(2,2))(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(200,3)(x)
    x = layers.Conv2D(200,3)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Conv2D(300,3,strides=(2,2))(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(300,3)(x)
    x = layers.Conv2D(300,3)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Conv2D(400,3,strides=(2,2))(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(400,3)(x)
    x = layers.Conv2D(400,3)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(256)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.1)(x)
    
    x = layers.Dense(64)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.1)(x)
 
    output = layers.Dense(1,activation="sigmoid")(x)
    model = tf.keras.Model(inputs=inputa, outputs=output, name="discriminator_model")
    return model
 
 
with strategy.scope():
    generator = make_generator_model()
    #generator.summary()
 
    discriminator = make_discriminator_model()
    #discriminator.summary()
 
if(resume):
    generator.load_weights(gen_weights)
    discriminator.load_weights(disc_weights)
    

In [ ]:
generator_optimizer =   tf.keras.optimizers.Adam(0.0000005)
disc_optimizer =        tf.keras.optimizers.Adam(0.0000005)
 
#generator_optimizer =  tf.keras.optimizers.Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07)
#disc_optimizer =       tf.keras.optimizers.Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07)mse = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM)

In [ ]:
mse = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM)
 
 
def _gaussian_kernel(kernel_size, sigma, n_channels, dtype):
        x = tf.range(-kernel_size // 2 + 1, kernel_size // 2 + 1, dtype=dtype)
        g = tf.math.exp(-(tf.pow(x, 2) / (2 * tf.pow(tf.cast(sigma, dtype), 2))))
        g_norm2d = tf.pow(tf.reduce_sum(g), 2)
        g_kernel = tf.tensordot(g, g, axes=0) / g_norm2d
        g_kernel = tf.expand_dims(g_kernel, axis=-1)
        return tf.expand_dims(tf.tile(g_kernel, (1, 1, n_channels)), axis=-1)
 
 
def apply_blur(img):
        blur = _gaussian_kernel(3, 2, 3, img.dtype)
        img = tf.nn.depthwise_conv2d(img, blur, [1,1,1,1], 'SAME')
        return img
 
def calc_discriminator_loss(true_output,fake_output):
    total_loss = mse(tf.ones_like(true_output),true_output)+mse(tf.zeros_like(fake_output),fake_output)
 
  
    return total_loss
 
 
def calc_generator_loss(genr,norm, disc_output):
        normb=apply_blur(norm)
        genrb=apply_blur(genr)
        #pploss=tf.math.reduce_sum((normb-genrb)*(normb-genrb)/(128*128*3))
        pploss=tf.math.reduce_sum((normb-genrb)*(normb-genrb)/(128*128*3)+(norm-genr)*(norm-genr)/(128*128*3))
 
        dloss=mse(tf.ones_like(disc_output),disc_output)
        total_loss=dloss+pploss*2
 
        return total_loss
 
 
 
@tf.function
def train_step_cartoon(cartoons):
        with tf.GradientTape() as tape, tf.GradientTape() as dtape:
            generated_images = generator(cartoons, training=True)
 
            noise1 = np.random.normal(0, .05, generated_images.shape)
            noise2 = np.random.normal(0, .05, cartoons.shape)
 
            generated_images_noise = generated_images + noise1
            cartoons_noise = cartoons + noise2
            
            d_true = discriminator(cartoons_noise,training=True)
            d_false = discriminator(generated_images_noise,training=True)
 
            
            discriminator_loss = calc_discriminator_loss(d_true,d_false)
            gen_loss = calc_generator_loss(generated_images,cartoons,d_false)
            
      
        gradients_of_generator = tape.gradient(gen_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
 
        gradients_of_disc = dtape.gradient(discriminator_loss, discriminator.trainable_variables)
        disc_optimizer.apply_gradients(zip(gradients_of_disc, discriminator.trainable_variables))
      
        
def train(dataset_cartoon):
    #x=Thread(target=load_next)
    #x.daemon=True
    #x.start()
    load_next()
    for i in range(5):
        for seed_batch in dataset_cartoon:
            strategy.run(train_step_cartoon,args=(seed_batch,))
 
test_imgs = np.array(test_imgs)
 
    
fig = plt.figure(figsize=(100,10))
for i in range(test_imgs.shape[0]):
        img = cv2.cvtColor(test_imgs[i],cv2.COLOR_BGR2RGB)
        plt.subplot(1, 10, i+1)
        plt.imshow((img+1)/2)
plt.show()
 
def generate_images():    
    predictions = generator(test_imgs)
    generated_images = np.array(predictions,dtype= 'float32')
    fig = plt.figure(figsize=(100,10))
    for i in range(generated_images.shape[0]):
        img = cv2.cvtColor(generated_images[i],cv2.COLOR_BGR2RGB)
        plt.subplot(1, 10, i+1)
        plt.imshow((img+1)/2)
    plt.show()
 
    
generate_images()

In [ ]:
tzsl = 0
while(True):
    t=train_images_cartoon
   
    
    train(t)
    tzsl+=1
    if(tzsl%3==0):
      generate_images() 
      generator.save_weights('/content/drive/MyDrive/gen_weights.h5', overwrite=True)
      discriminator.save_weights('/content/drive/MyDrive/disc_weights.h5', overwrite=True)

In [ ]:
generator.save_weights('/content/drive/MyDrive/gen_weights.h5', overwrite=True)
discriminator.save_weights('/content/drive/MyDrive/disc_weights.h5', overwrite=True)